In [7]:
import pandas as pd
import numpy as np

In [14]:
!pip install sentence_transformers

In [16]:
!pip install -U huggingface_hub

     |████████████████████████████████| 199 kB 7.1 MB/s eta 0:00:01
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.0.8
    Uninstalling huggingface-hub-0.0.8:
      Successfully uninstalled huggingface-hub-0.0.8
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
transformers 4.6.1 requires huggingface-hub==0.0.8, but you have huggingface-hub 0.13.0 which is incompatible.
datasets 2.1.0 requires tqdm>=4.62.1, but you have tqdm 4.61.2 which is incompatible.


In [9]:
from config import Config
Config.switch_env("prod")
import pandas as pd
from util.read_write_data_azure_blob import ReadAndWriteFromAzureBlob
from datetime import datetime, timedelta
from relevance.keywordtargeting import KeywordTargeting

ImportError: cannot import name 'DatasetInfo' from 'huggingface_hub.hf_api' (/opt/anaconda3/lib/python3.8/site-packages/huggingface_hub/hf_api.py)

In [3]:
class get_search_volume_or_atc:
    def __init__(self):
        self.read_write_blob = ReadAndWriteFromAzureBlob()
    def get_lastn_days_job_data(self, retailer_id, job_type, ndays=10):
        if job_type == 'atc':
            job_name = 'OrganicProductKeywordMap.csv'
            data_path_string = 'retailer_id={retailer_id}/year={year}/month={month}/day={day}/{job_name}'
            connection_string = Config.AzureConfig.ICC_DS_STORAGE_CONNECTION_STRING
            container_name = 'combined-output'
            read_flag = False
        elif job_type == 'organic_search':
            job_name = 'keyword_count'
            data_path_string = 'organic_data/retailer_id={retailer_id}/year={year}/month={month}/day={day}/job={job_name}/'
            connection_string = Config.AzureConfig.SUGGESTIONS_INPUT_CONNECTION_STRING
            container_name = Config.AzureConfig.SUGGESTION_INPUT_CONTAINER
            read_flag = True
        elif job_type == 'organic_sales':
            job_name = 'place_order_data'
            data_path_string = 'organic_data/retailer_id={retailer_id}/year={year}/month={month}/day={day}/job={job_name}/'
            connection_string = Config.AzureConfig.SUGGESTIONS_INPUT_CONNECTION_STRING
            container_name = Config.AzureConfig.SUGGESTION_INPUT_CONTAINER
            read_flag = True
        job_nday_data = pd.DataFrame()
        for day in range(1, ndays + 1):
            try:
                date_string = datetime.strftime(datetime.now() - timedelta(day), '%Y/%m/%d')
                year, month, day = date_string.split("/")
                blob_path = data_path_string.format(retailer_id=retailer_id, year=year, month=month, day=day,
                                                    job_name=job_name)
                print(blob_path)
                job_data_daily = self.read_write_blob.find_blob_name_from_container_and_read_df(connection_string,
                                                                                                container_name,
                                                                                                [blob_path],
                                                                                                do_csv_check=read_flag)
                job_data_daily['Date'] = date_string
                job_nday_data = pd.concat([job_nday_data, job_data_daily])
            except Exception as e:
                print(e)
                continue
        if job_type == 'atc':
            job_nday_data.rename(columns={'Keyword': 'keywords'}, inplace=True)
        elif job_type == 'organic_search':
            job_nday_data.rename(columns={'keyword': 'keywords'}, inplace=True)
        return job_nday_data

In [6]:
get_search_volume_or_atc.get_lastn_days_job_data('123456','atc',2)

local variable 'data_path_string' referenced before assignment
local variable 'data_path_string' referenced before assignment
local variable 'data_path_string' referenced before assignment
local variable 'data_path_string' referenced before assignment
local variable 'data_path_string' referenced before assignment
local variable 'data_path_string' referenced before assignment
local variable 'data_path_string' referenced before assignment
local variable 'data_path_string' referenced before assignment
local variable 'data_path_string' referenced before assignment
local variable 'data_path_string' referenced before assignment


Empty DataFrame
Columns: []
Index: []